<a href="https://colab.research.google.com/github/drdosan/cap1-despertar-da-rede-neural/blob/main/document/DiogoRebello_rm565286_pbl_fase6_Otimizado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detecção de Vestuário com YOLOv5
Este trabalho tem como objetivo treinar uma rede YOLOv5 para detectar duas peças de vestuário distintas: **blusas** e **sapatos**.
O dataset foi construído com 40 imagens de cada classe (80 no total), rotuladas manualmente no MakeSense.

## Montando Google Drive
Conectamos o Colab ao Google Drive para acessar as imagens, labels e salvar os resultados do treino.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

BASE = "/content/drive/MyDrive/roupas"
ORIG = f"{BASE}/_originais"

!find /content/drive/MyDrive/ -type f -name "*.cache" -delete
!find /content/yolov5/ -type f -name "*.cache" -delete

## Organização do Dataset
- 40 imagens de blusas e 40 de sapatos.
- Cada imagem rotulada com bounding boxes no formato YOLO.
- Divisão:
  - 32 imagens por classe → treino
  - 4 imagens por classe → validação
  - 4 imagens por classe → teste
O código abaixo organiza as pastas automaticamente.


In [ ]:
import os

for kind in ["images","labels"]:
    for split in ["train","val","test"]:
        for cls in ["blusas","sapatos"]:
            os.makedirs(f"{BASE}/{kind}/{split}/{cls}", exist_ok=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import glob, random, shutil, os

random.seed(42)

def move_pair(img_path, dst_img_dir, dst_lbl_dir):
    name = os.path.splitext(os.path.basename(img_path))[0]
    lbl_path = os.path.join(os.path.dirname(img_path), f"{name}.txt")
    # destino
    shutil.copy2(img_path, os.path.join(dst_img_dir, os.path.basename(img_path)))
    if os.path.exists(lbl_path):
        shutil.copy2(lbl_path, os.path.join(dst_lbl_dir, os.path.basename(lbl_path)))
    else:
        print("⚠️ sem label:", img_path)

for cls in ["blusas","sapatos"]:
    imgs = sorted(sum([glob.glob(f"{ORIG}/{cls}/*.{ext}") for ext in ["jpg","jpeg","png","JPG","JPEG","PNG"]], []))
    assert len(imgs) >= 40, f"Classe {cls} tem menos de 40 imagens"
    random.shuffle(imgs)

    train_imgs = imgs[:32]
    val_imgs   = imgs[32:36]
    test_imgs  = imgs[36:40]

    for p in train_imgs:
        move_pair(p, f"{BASE}/images/train/{cls}", f"{BASE}/labels/train/{cls}")
    for p in val_imgs:
        move_pair(p, f"{BASE}/images/val/{cls}", f"{BASE}/labels/val/{cls}")
    for p in test_imgs:
        move_pair(p, f"{BASE}/images/test/{cls}", f"{BASE}/labels/test/{cls}")

sum_train = len(glob.glob(f"{BASE}/images/train/*/*"))
sum_val   = len(glob.glob(f"{BASE}/images/val/*/*"))
sum_test  = len(glob.glob(f"{BASE}/images/test/*/*"))
print("imgs -> train/val/test:", sum_train, sum_val, sum_test)

In [ ]:
import glob, collections

BASE = "/content/drive/MyDrive/roupas/labels"

def contar_ids(split, cls):
    c = collections.Counter(); vazios = 0
    for p in glob.glob(f"{BASE}/{split}/{cls}/*.txt"):
        with open(p) as f:
            txt = f.read().strip()
        if not txt:
            vazios += 1
            continue
        for line in txt.splitlines():
            t = line.strip().split()
            if t:
                c[t[0]] += 1
    return c, vazios

for split in ["train","val","test"]:
    c0,v0 = contar_ids(split,"blusas")
    c1,v1 = contar_ids(split,"sapatos")
    print(f"[{split}] blusas -> IDs {c0} | vazios: {v0}")
    print(f"[{split}] sapatos   -> IDs {c1} | vazios: {v1}")



In [ ]:
# 🔄 Criar/atualizar pasta test_infer (sem subpastas)
import os, glob, shutil
BASE = "/content/drive/MyDrive/roupas"
TEST_DIR = f"{BASE}/images/test"
TEST_INFER = f"{BASE}/images/test_infer"
os.makedirs(TEST_INFER, exist_ok=True)
for f in glob.glob(f"{TEST_INFER}/*.*"): os.remove(f)
for ext in ("*.jpg","*.jpeg","*.png","*.JPG","*.JPEG","*.PNG"):
    for sub in ("blusas","sapatos"):
        for p in glob.glob(f"{TEST_DIR}/{sub}/{ext}"):
            shutil.copy2(p, os.path.join(TEST_INFER, os.path.basename(p)))
print("OK test_infer")

## Preparação do YOLOv5
Clonamos o repositório oficial do YOLOv5 e instalamos as dependências necessárias.


In [ ]:
%cd /content
!git clone -q https://github.com/ultralytics/yolov5
%cd /content/yolov5
!pip install -q -r requirements.txt

## Treinamento - 100 épocas
Primeira simulação com 100 épocas para observar o desempenho inicial do modelo.


In [ ]:
# 🔧 Hiperparâmetros completos p/ YOLOv5 (base scratch-low + ajustes talheres)
hyp_text = """
# Optimizer
lr0: 0.01
lrf: 0.01
momentum: 0.937
weight_decay: 0.0005
warmup_epochs: 3.0
warmup_momentum: 0.8
warmup_bias_lr: 0.1

# Loss
box: 0.05
cls: 1.5
cls_pw: 1.0
obj: 1.0
obj_pw: 1.0
label_smoothing: 0.0
fl_gamma: 0.0

# Targets / anchors
iou_t: 0.20
anchor_t: 4.0

# Augmentations
hsv_h: 0.015
hsv_s: 0.7
hsv_v: 0.4
degrees: 10.0
translate: 0.20
scale: 0.50
shear: 2.0
perspective: 0.0
flipud: 0.0
fliplr: 0.5
mosaic: 1.0
mixup: 0.10
copy_paste: 0.10
"""
path_hyp = "/content/yolov5/data/hyps/hyp_roupas.yaml"
with open(path_hyp, "w") as f:
    f.write(hyp_text)
print("✅ Criado:", path_hyp)


In [ ]:
DATA_YAML = "/content/drive/MyDrive/roupas/data.yaml"

RUN_100 = "roupas_100ep"
!python train.py --img 832 --batch 16 --epochs 100 \
  --data "{DATA_YAML}" --weights yolov5s.pt \
  --hyp data/hyps/hyp_roupas.yaml \
  --project "/content/drive/MyDrive/roupas/runs" \
  --name "{RUN_100}" --exist-ok


In [ ]:
import os, glob, random
from IPython.display import Image, display

def mostrar_resultados(pasta, n=3):
    exts = ("*.jpg","*.jpeg","*.png","*.JPG","*.JPEG","*.PNG")
    imgs_train_100_epoch = []
    for e in exts:
        imgs_train_100_epoch += glob.glob(os.path.join(pasta, e))
    if not imgs_train_100_epoch:
        crop_dir = os.path.join(pasta, "crops")
        for e in exts:
            imgs_train_100_epoch += glob.glob(os.path.join(crop_dir, "**", e), recursive=True)
    if not imgs_train_100_epoch:
        print(f"Nenhuma imagem encontrada em {pasta}")
        return
    for img in imgs_train_100_epoch:
        display(Image(filename=img))

print("🔹 Treinamento - 100 épocas")
mostrar_resultados("/content/drive/MyDrive/roupas/runs/roupas_100ep")

In [ ]:
import torch

BEST_100 = f"/content/drive/MyDrive/roupas/runs/{RUN_100}/weights/best.pt"
!ls -l "{BEST_100}"

!python val.py \
  --weights "{BEST_100}" \
  --data "{DATA_YAML}" \
  --img 832 \
  --save-json \
  --save-hybrid \
  --project "/content/drive/MyDrive/roupas/runs/val" \
  --name "exp_best100" \
  --exist-ok



In [ ]:
import os, glob, random
from IPython.display import Image, display

def mostrar_resultados(pasta, n=3):
    exts = ("*.jpg","*.jpeg","*.png","*.JPG","*.JPEG","*.PNG")
    imgs_val_100_epoch = []
    for e in exts:
        imgs_val_100_epoch += glob.glob(os.path.join(pasta, e))
    if not imgs_val_100_epoch:
        crop_dir = os.path.join(pasta, "crops")
        for e in exts:
            imgs_val_100_epoch += glob.glob(os.path.join(crop_dir, "**", e), recursive=True)
    if not imgs_val_100_epoch:
        print(f"Nenhuma imagem encontrada em {pasta}")
        return
    for img in imgs_val_100_epoch:
        display(Image(filename=img))

print("🔹 Validação - 100 épocas")
mostrar_resultados("/content/drive/MyDrive/roupas/runs/val/exp_best100")

### Resultados - 100 épocas
O bloco abaixo mostra métricas de validação e gera predições em imagens de teste.
As saídas ficam em `/runs/detect/exp_30`.


In [ ]:
!python detect.py --weights "{BEST_100}" --data "{DATA_YAML}" --conf 0.15 --img 832 \
  --source "{TEST_INFER}" \
  --project "/content/drive/MyDrive/roupas/runs/detect" \
  --name "exp_100_fix" --exist-ok --line-thickness 2

In [ ]:
import os, glob, random
from IPython.display import Image, display

def mostrar_resultados(pasta, n=3):
    exts = ("*.jpg","*.jpeg","*.png","*.JPG","*.JPEG","*.PNG")
    imgs_detect_100_epoch = []
    for e in exts:
        imgs_detect_100_epoch += glob.glob(os.path.join(pasta, e))
    if not imgs_detect_100_epoch:
        crop_dir = os.path.join(pasta, "crops")
        for e in exts:
            imgs_detect_100_epoch += glob.glob(os.path.join(crop_dir, "**", e), recursive=True)
    if not imgs_detect_100_epoch:
        print(f"Nenhuma imagem encontrada em {pasta}")
        return
    for img in imgs_detect_100_epoch:
        display(Image(filename=img))

print("🔹 Detecção - 100 épocas")
mostrar_resultados("/content/drive/MyDrive/roupas/runs/detect/exp_100_fix")

## Treinamento - 200 épocas
Segunda simulação, agora com 200 épocas, para avaliar a evolução do desempenho.


In [ ]:
RUN_200 = "roupas_200ep"
!python train.py --img 832 --batch 16 --epochs 200 \
  --data "{DATA_YAML}" --weights yolov5s.pt \
  --project "/content/drive/MyDrive/roupas/runs" \
  --name "{RUN_200}" --exist-ok

In [ ]:
import os, glob, random
from IPython.display import Image, display

def mostrar_resultados(pasta, n=3):
    exts = ("*.jpg","*.jpeg","*.png","*.JPG","*.JPEG","*.PNG")
    imgs_train_200_epoch = []
    for e in exts:
        imgs_train_200_epoch += glob.glob(os.path.join(pasta, e))
    if not imgs_train_200_epoch:
        crop_dir = os.path.join(pasta, "crops")
        for e in exts:
            imgs_train_200_epoch += glob.glob(os.path.join(crop_dir, "**", e), recursive=True)
    if not imgs_train_200_epoch:
        print(f"Nenhuma imagem encontrada em {pasta}")
        return
    for img in imgs_train_200_epoch:
        display(Image(filename=img))

print("🔹 Treinamento - 200 épocas")
mostrar_resultados("/content/drive/MyDrive/roupas/runs/roupas_200ep")

### Resultados - 200 épocas
Avaliação final e imagens de teste disponíveis em `/runs/detect/exp_200`.


In [ ]:
BEST_200 = f"/content/drive/MyDrive/roupas/runs/{RUN_200}/weights/best.pt"
!ls -l "{BEST_200}"

!python val.py \
  --weights "{BEST_200}" \
  --data "{DATA_YAML}" \
  --img 832 \
  --save-json \
  --save-hybrid \
  --project "/content/drive/MyDrive/roupas/runs/val" \
  --name "exp_best200" \
  --exist-ok

In [ ]:
import os, glob, random
from IPython.display import Image, display

def mostrar_resultados(pasta, n=3):
    exts = ("*.jpg","*.jpeg","*.png","*.JPG","*.JPEG","*.PNG")
    imgs_val_200_epoch = []
    for e in exts:
        imgs_val_200_epoch += glob.glob(os.path.join(pasta, e))
    if not imgs_val_200_epoch:
        crop_dir = os.path.join(pasta, "crops")
        for e in exts:
            imgs_val_200_epoch += glob.glob(os.path.join(crop_dir, "**", e), recursive=True)
    if not imgs_val_200_epoch:
        print(f"Nenhuma imagem encontrada em {pasta}")
        return
    for img in imgs_val_200_epoch:
        display(Image(filename=img))

print("🔹 Validação - 200 épocas")
mostrar_resultados("/content/drive/MyDrive/roupas/runs/val/exp_best200")

In [ ]:
!python detect.py --weights "{BEST_200}" --data "{DATA_YAML}" --conf 0.15 --img 832 \
  --source "{TEST_INFER}" \
  --project "/content/drive/MyDrive/roupas/runs/detect" \
  --name "exp_200_fix" --exist-ok --line-thickness 2

In [ ]:
import os, glob, random
from IPython.display import Image, display

def mostrar_resultados(pasta, n=3):
    exts = ("*.jpg","*.jpeg","*.png","*.JPG","*.JPEG","*.PNG")
    imgs_detect_200_epoch = []
    for e in exts:
        imgs_detect_200_epoch += glob.glob(os.path.join(pasta, e))
    if not imgs_detect_200_epoch:
        crop_dir = os.path.join(pasta, "crops")
        for e in exts:
            imgs_detect_200_epoch += glob.glob(os.path.join(crop_dir, "**", e), recursive=True)
    if not imgs_detect_200_epoch:
        print(f"Nenhuma imagem encontrada em {pasta}")
        return
    for img in imgs_detect_200_epoch:
        display(Image(filename=img))

print("🔹 Validação - 200 épocas")
mostrar_resultados("/content/drive/MyDrive/roupas/runs/detect/exp_200_fix")

## Comparação e Conclusões
- Comparação entre 100 e 200 épocas (mAP, precisão, recall).
- Análise do tempo de treino e inferência.
- Pontos fortes: modelo distinguiu bem blusas e sapatos.
- Limitações: dataset pequeno (80 imagens).
- Próximos passos: mais imagens, data augmentation, testar yolov5s/m.
